In [104]:
import time
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
from IPython.display import display, HTML
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
################################################################################# dong 이름 바꾸고 url 링크 바꾸기
dong = '양동'
################################################################################# # 덕흥동 매물 없음
lst = [304384.015, 3888182.041]

In [ ]:
s = Service(ChromeDriverManager().install())
driver = wb.Chrome(service=s)
driver.maximize_window()
# driver.get('https://new.land.naver.com/offices?ms=35.164324,126.88035,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL') # 광천동
# driver.get('https://new.land.naver.com/offices?ms=35.1318,126.8539,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL') # 금호동
# driver.get('https://new.land.naver.com/offices?ms=35.159,126.8729,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL') # 내방동
# driver.get('https://new.land.naver.com/offices?ms=35.154674,126.88945,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL') # 농성동
# driver.get('https://new.land.naver.com/offices?ms=35.171306,126.864736,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL') # 동천동
# driver.get('https://new.land.naver.com/offices?ms=35.1428,126.8418,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL') # 마륵동
driver.get('https://new.land.naver.com/offices?ms=35.1428,126.8418,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL') # 매월동


In [ ]:
columns = ['동', '위도', '경도', '상가종류', '월세', '평수', '층수']
df = pd.DataFrame(columns=columns)

map_clusters = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
result = []

time.sleep(3)
for i in range(len(map_clusters)):

    map_element = driver.find_element(By.CSS_SELECTOR, "div#map.map_panel")
    center_x = map_element.size['width'] / 2
    center_y = map_element.size['height'] / 2

    time.sleep(1)
    map_clusters = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
    target_x = float(map_clusters[i].get_attribute("style").split(" ")[-1].replace("px;", ""))
    target_y = float(map_clusters[i].get_attribute("style").split(" ")[-3].replace("px;", ""))

    delta_x = target_x - center_x
    delta_y = target_y - center_y

    distance_x = delta_x * 100 / 52
    distance_y = delta_y * 100 / 52

    print(f"화면 크기 : x : {map_element.size['width']}, y : {map_element.size['height']}")
    print(f"타깃 좌표 : x : {target_x}, y : {target_y}")
    utm_coordinates = f"52 N {int(lst[0]) + distance_x} {int(lst[1]) - distance_y}"
    print(utm_coordinates)
    
    url = driver.current_url  # 먼저 문자열로 저장
    coord_str = url[url.find("ms=")+3 : url.find(",18&a")]  # "ms=" 다음부터 좌표 추출
    coords = coord_str.split(',')  # ['위도', '경도']
    
    map_clusters[i].click()

    
    try:
        body = driver.find_element(By.CSS_SELECTOR, 'div.item_list')

        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            body.send_keys(Keys.END)

            time.sleep(1)

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        body.send_keys(Keys.END)
    except:
        pass

    target_items = driver.find_elements(By.CSS_SELECTOR, "div.item_title")
    for item in target_items:

        item.click()
        time.sleep(0.5)

        ty = driver.find_elements(By.CSS_SELECTOR, "h4.info_title")
        title = ty[1].text

        pay = driver.find_element(By.CSS_SELECTOR, "div.info_article_price ")
        price = pay.text

        rows = driver.find_elements(By.CSS_SELECTOR, "tr.info_table_item")

        data = {}
        stop_flag = False
        for row in rows:
            if stop_flag:
                break
            ths = row.find_elements(By.CSS_SELECTOR, "th")
            tds = row.find_elements(By.CSS_SELECTOR, "td")

            for i in range(len(ths)):
                key = ths[i].text.strip()
                value = tds[i].text.strip() if i < len(tds) else ""

                data[key] = value

                if key == "건물번호":
                    stop_flag = True
                    break

        # 제목과 가격 추가
        data["제목"] = title
        data["가격"] = price
        data['위도'] = coords[0]
        data['경도'] = coords[1]
        
        df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
        unique_coords = df[['위도', '경도']].drop_duplicates()

    c_btns = driver.find_elements(By.CSS_SELECTOR, "button.btn_close")

    for btn in c_btns:
        label = btn.get_attribute("aria-label")
        if label == "상세페이지 닫기":
            btn.click()
            break  # 클릭한 후 반복 종료
    time.sleep(2)
    print(i)
for i in range(len(result)):
    df.loc[i] = result[i]



화면 크기 : x : 2005, y : 1089
타깃 좌표 : x : 760.685, y : 495.597
52 N 302921.9711538461 3891105.0442307694
0
화면 크기 : x : 2005, y : 1089
타깃 좌표 : x : 914.449, y : 510.358
52 N 303217.67115384614 3891076.6576923076
1
화면 크기 : x : 2005, y : 1089
타깃 좌표 : x : 594.731, y : 361.492
52 N 302602.82884615386 3891362.9384615384
2
화면 크기 : x : 2005, y : 1089
타깃 좌표 : x : 805.712, y : 359.346
52 N 303008.56153846154 3891367.065384615
3
화면 크기 : x : 2005, y : 1089
타깃 좌표 : x : 638.118, y : 628.957
52 N 302686.2653846154 3890848.582692308
4
화면 크기 : x : 2005, y : 1089
타깃 좌표 : x : 703.742, y : 592.698
52 N 302812.4653846154 3890918.3115384616
5
화면 크기 : x : 2005, y : 1089
타깃 좌표 : x : 758.868, y : 592.426
52 N 302918.4769230769 3890918.8346153847
6
화면 크기 : x : 2005, y : 1089
타깃 좌표 : x : 1054.68, y : 450.242
52 N 303487.3461538461 3891192.2653846154
7
화면 크기 : x : 2005, y : 1089
타깃 좌표 : x : 1204.58, y : 453.537
52 N 303775.6153846154 3891185.928846154
8
화면 크기 : x : 2005, y : 1089
타깃 좌표 : x : 1359.56, y : 483.942
52 N

In [117]:
time.sleep(1)
driver.execute_script("window.open('https://www.movable-type.co.uk/scripts/latlong-utm-mgrs.html');")
time.sleep(1)
driver.switch_to.window(driver.window_handles[-1])
time.sleep(2)

driver.find_element(By.CSS_SELECTOR, "input#deg-format-d").click()
driver.find_element(By.CSS_SELECTOR, "input#precision-mm").click()
time.sleep(1)

for i in range(df.shape[0]):
    # UTM 좌표 입력 필드 가져오기
    utm_input = driver.find_element(By.CSS_SELECTOR, "input#utm-coord")

    # 기존 값 삭제 후 새로운 UTM 좌표 입력
    utm_input.clear()
    utm_input.send_keys("52 N " + str(df.loc[i, '위도']) + ' ' + str(df.loc[i, '경도']))
    utm_input.send_keys(Keys.RETURN)
    time.sleep(0.1)

    # 변환된 위도/경도 가져오기
    latlon_value = driver.find_element(By.CSS_SELECTOR, "input#latlon").get_attribute("value")

    latlon_value.find(", ")
    lat, long = latlon_value.split(", ")
    df.loc[i, '위도'] = float(lat[:-3])
    df.loc[i, '경도'] = float(long[:-3])


In [118]:
df.to_csv(f"{dong}_부동산.csv", encoding='utf-8-sig', index=False)